In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
cd drive/My Drive/DL_3/

In [0]:
!pip install tensorboardx

In [0]:
import random
import numpy as np
from itertools import cycle
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torch.autograd import Variable
from torch.utils.data import DataLoader
import h5py
from itertools import cycle
from collections import OrderedDict
from scipy.io import loadmat
import glob
import os
import torch.optim as optim
from torch.autograd import Variable
from tensorboardX import SummaryWriter
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch.autograd import Variable
from mpl_toolkits.axes_grid1 import ImageGrid
from torchvision.transforms import Compose, ToTensor
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from collections import defaultdict


In [0]:
filenames = glob.glob('data/sprites/*.mat')
filenames = sorted(filenames)
split_file = filenames[len(filenames) - 1]
print(split_file)
filenames = filenames[0 : len(filenames) - 2]
print(filenames)

In [0]:
print(split_file)
f = loadmat(split_file)
print(f.keys())
print(f['trainidx'][0])
print(f['testidx'][0])
print(f['validx'][0])
# print(f['sprites'].shape)
# for j in range(0, 21):
#   print(np.shape(f[f.get('sprites')[j][0]].value))


In [0]:
train_dict = defaultdict()
train_dict = { i : 1 for i in f['trainidx'][0]}
print(train_dict)
test_dict = defaultdict()
test_dict = {i : 1 for i in f['testidx'][0]}
print(test_dict)
valid_dict = defaultdict()
valid_dict = {i : 1 for i in f['validx'][0]}
print(valid_dict)

In [0]:
train_filenames = []
test_filenames = []
valid_filenames = []
file_list = []
c = 0
for i in filenames:
  str = i[-7: -4]
  # print(str)
  if str[0] >= 'a' and str[0] <= 'z':
    # print(str[2:])
    str = int(str[2:])
  elif str[0] == '_':
    # print(str[1:])
    str = int(str[1 :])
  else:
    str = int(str)
  # print(str)
  file_list.append(str)

for i in file_list:
  if i in train_dict:
    train_filenames.append(filenames[c])
  elif i in test_dict:
    test_filenames.append(filenames[c])
  elif i in valid_dict:
    valid_filenames.append(filenames[c])
  else:
    print(type(i), type(train_dict[585]))
  c += 1
print(c)
print(len(train_filenames))
print(len(test_filenames))
print(len(valid_filenames))
  

In [0]:
def construct_dictionary(filenames):
  num = 0
  data_dict = {}
  for i in filenames[0: 100]:
    f = h5py.File(i, 'r')
    # print(np.shape(f.get('sprites')))
    # arr = np.reshape(f[f.get('sprites')[4][0]].value,(7, 3, 60, 60))
    for j in range(0, 21):
      # print(np.shape(f[f.get('sprites')[j][0]].value))
      k = np.shape(f[f.get('sprites')[j][0]].value)[0]
      arr = np.reshape(f[f.get('sprites')[j][0]].value,(k, 3, 60, 60))
      if num not in data_dict:
        data_dict[num] =  arr
      else:
        arr2 = data_dict[num]
        arr2 = np.concatenate([arr2, arr], 0)
        data_dict[num] = arr2

    num += 1
  return data_dict

In [0]:
data_dict = construct_dictionary(train_filenames)

In [0]:
test_data_dict = construct_dictionary(test_filenames)

In [0]:
# for keys in data_dict:
#   # print(keys)
#   print(np.shape(data_dict[keys]))


In [0]:
import torch
from torch.utils import data

class Dataset():
  'Characterizes a dataset for PyTorch'
  def __init__(self, data_dict, num_samples):
        'Initialization'
        self.data_dict = data_dict
        self.num_samples = num_samples
  def __len__(self):
        'Denotes the total number of samples'
        return self.num_samples

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        key = (index // 178) 
        num_img = (index // 100)
        # print(key)
        arr = self.data_dict[key]
        image = arr[num_img]
        label = key
        return image, random.SystemRandom().choice(self.data_dict[key]), label

dataset = Dataset(data_dict, 178 * 100)
loader = cycle(DataLoader(dataset, batch_size = 8, shuffle = True, num_workers=0, drop_last=True))
print(dataset.data_dict.keys())

image_batch, image_batch_2, labels_batch = next(loader)
print(labels_batch)

In [0]:
class test_Dataset():
  'Characterizes a dataset for PyTorch'
  def __init__(self, data_dict, num_samples):
        'Initialization'
        self.data_dict = data_dict
        self.num_samples = num_samples
  def __len__(self):
        'Denotes the total number of samples'
        return self.num_samples

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        key = (index // 178) 
        num_img = (index // 100)
        # print(key)
        arr = self.data_dict[key]
        image = arr[num_img]
        label = key
        return image, random.SystemRandom().choice(self.data_dict[key]), label

test_dataset = test_Dataset(test_data_dict, 178 * 100)
test_loader = cycle(DataLoader(test_dataset, batch_size = 1, shuffle = True, num_workers=0, drop_last=True))
print(dataset.data_dict.keys())

image_batch, image_batch_2, labels_batch = next(test_loader)
print(labels_batch)

In [0]:
image_batch_2.size()

In [0]:
# Architecture taken from @ https://github.com/ananyahjha93/cycle-consistent-vae
class Encoder(nn.Module):
    def __init__(self, style_dim, class_dim):
        super(Encoder, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels= 3, out_channels=16, kernel_size=5, stride=2, padding=1, bias=True)
        self.conv_1_in = nn.InstanceNorm2d(num_features=16, track_running_stats=True)
        self.relu_1 = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=2, padding=1, bias=True)
        self.conv_2_in = nn.InstanceNorm2d(num_features=32, track_running_stats=True)
        self.relu_2 = nn.ReLU(inplace=True)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=2, padding=1, bias=True)
        self.conv_3_in = nn.InstanceNorm2d(num_features=64, track_running_stats=True)
        self.relu_3 = nn.ReLU(inplace=True)

        # Style embeddings
        self.style_mu = nn.Linear(in_features= 2304, out_features=style_dim, bias=True)
        self.style_logvar = nn.Linear(in_features= 2304, out_features=style_dim, bias=True)

        # Class embeddings
        self.class_output = nn.Linear(in_features=2304, out_features=class_dim, bias=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv_1_in(x)
        x = self.relu_1(x)
        x = self.conv2(x)
        x = self.conv_2_in(x)
        x = self.relu_2(x)
        x = self.conv3(x)
        x = self.conv_3_in(x)
        x = self.relu_3(x)
        x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))

        style_embeddings_mu = self.style_mu(x)
        style_embeddings_logvar = self.style_logvar(x)
        class_embeddings = self.class_output(x)

        return style_embeddings_mu, style_embeddings_logvar, class_embeddings

class Decoder(nn.Module):
    def __init__(self, style_dim, class_dim):
        super(Decoder, self).__init__()

        # Style embeddings input
        self.style_input = nn.Linear(in_features=style_dim, out_features=2304, bias=True)

        # Class embeddings input
        self.class_input = nn.Linear(in_features=class_dim, out_features=2304, bias=True)

        self.deconv1 = nn.ConvTranspose2d(in_channels=128, out_channels=32, kernel_size=4, stride=2, padding=0, bias=True)
        self.deconv1_in = nn.InstanceNorm2d(num_features=32, track_running_stats=True)
        self.deconv1_relu = nn.LeakyReLU(negative_slope=0.2, inplace=True)

        self.deconv2 = nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=4, stride=2, padding=0, bias=True)
        self.deconv2_in = nn.InstanceNorm2d(num_features= 16, track_running_stats=True)
        self.deconv2_relu = nn.LeakyReLU(negative_slope=0.2, inplace=True)

        self.deconv3 = nn.ConvTranspose2d(in_channels=16, out_channels= 3, kernel_size=4, stride=2, padding=1, bias=True)
        self.sigmoid_output = nn.Sigmoid()
       
    def forward(self, style_embeddings, class_embeddings):
        style_embeddings = F.leaky_relu_(self.style_input(style_embeddings), negative_slope=0.2)
        class_embeddings = F.leaky_relu_(self.class_input(class_embeddings), negative_slope=0.2)

        x = torch.cat((style_embeddings, class_embeddings), dim=1)
        x = x.view(x.size(0), 128, 6, 6)
        x = self.deconv1(x)
        x = self.deconv1_in(x)
        x = self.deconv1_relu(x)

        x = self.deconv2(x)
        x = self.deconv2_in(x)
        x = self.deconv2_relu(x)

        x = self.deconv3(x)
        x = self.sigmoid_output(x)

        return x



In [0]:
class ags():
  def __init__(self):
 
# add arguments
  
    self.cuda = True
    self.batch_size = 16
    self.num_channels = 3
    self.initial_learning_rate = 0.0001
    self.style_dim = 64
    self.class_dim = 64
    self.num_classes = 500
    self.reconstruction_coef = 2
    self.reverse_cycle_coef = 10.
    self.kl_divergence_coef = 3.
    self.beta_1 = 0.9
    self.beta_2 = 0.999
    self.log_file = 'log.txt'
    self.image_size = 60
    self.load_saved = False
    self.encoder_save = 'encoder'
    self.decoder_save = 'decoder'
    self.start_epoch = 0
    self.end_epoch = 10

  
FLAGS = ags()


In [0]:
transform_config = Compose([ToTensor()])


def mse_loss(input, target):
    return torch.sum((input - target).pow(2)) / input.data.nelement()


def l1_loss(input, target):
    return torch.sum(torch.abs(input - target)) / input.data.nelement()


def reparameterize(training, mu, logvar):
    if training:
        std = logvar.mul(0.5).exp_()
        eps = Variable(std.data.new(std.size()).normal_())
        return eps.mul(std).add_(mu)
    else:
        return mu


def weights_init(layer):
    if isinstance(layer, nn.Conv2d):
        layer.weight.data.normal_(0.0, 0.05)
        layer.bias.data.zero_()
    elif isinstance(layer, nn.BatchNorm2d):
        layer.weight.data.normal_(1.0, 0.02)
        layer.bias.data.zero_()
    elif isinstance(layer, nn.Linear):
        layer.weight.data.normal_(0.0, 0.05)
        layer.bias.data.zero_()

def train(FLAGS):
    """
    model definition
    """
    encoder = Encoder(style_dim=FLAGS.style_dim, class_dim=FLAGS.class_dim)
    encoder.apply(weights_init)

    decoder = Decoder(style_dim=FLAGS.style_dim, class_dim=FLAGS.class_dim)
    decoder.apply(weights_init)

    
    

    style_latent_space = torch.FloatTensor(FLAGS.batch_size, FLAGS.style_dim)

    """
    loss definitions
    """
    cross_entropy_loss = nn.CrossEntropyLoss()

    '''
    add option to run on GPU
    '''
    if FLAGS.cuda:
        encoder.cuda()
        decoder.cuda()

        cross_entropy_loss.cuda()

        X_1 = X_1.cuda()
        X_2 = X_2.cuda()
        X_3 = X_3.cuda()

        style_latent_space = style_latent_space.cuda()

    """
    optimizer and scheduler definition
    """
    auto_encoder_optimizer = optim.Adam(
        list(encoder.parameters()) + list(decoder.parameters()),
        lr=FLAGS.initial_learning_rate,
        betas=(FLAGS.beta_1, FLAGS.beta_2)
    )

    reverse_cycle_optimizer = optim.Adam(
        list(encoder.parameters()),
        lr=FLAGS.initial_learning_rate,
        betas=(FLAGS.beta_1, FLAGS.beta_2)
    )

    # divide the learning rate by a factor of 10 after 80 epochs
    auto_encoder_scheduler = optim.lr_scheduler.StepLR(auto_encoder_optimizer, step_size=80, gamma=0.1)
    reverse_cycle_scheduler = optim.lr_scheduler.StepLR(reverse_cycle_optimizer, step_size=80, gamma=0.1)

    """
    training
    """
    if torch.cuda.is_available() and not FLAGS.cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

    if not os.path.exists('checkpoints'):
        os.makedirs('checkpoints')

    if not os.path.exists('reconstructed_images'):
        os.makedirs('reconstructed_images')

    # load_saved is false when training is started from 0th iteration
    if not FLAGS.load_saved:
        with open(FLAGS.log_file, 'w') as log:
            log.write('Epoch\tIteration\tReconstruction_loss\tKL_divergence_loss\tReverse_cycle_loss\n')

    # load data set and create data loader instance
    print('Loading stripes paired dataset...')
    
    loader = cycle(DataLoader(dataset, batch_size=FLAGS.batch_size, shuffle=True, num_workers=0, drop_last=True))

    # initialize summary writer
    writer = SummaryWriter()

    for epoch in range(FLAGS.start_epoch, FLAGS.end_epoch):
        print('')
        print('Epoch #' + str(epoch) + '..........................................................................')

        # update the learning rate scheduler
        auto_encoder_scheduler.step()
        reverse_cycle_scheduler.step()

        for iteration in range(int(len(dataset) / FLAGS.batch_size)):
            # A. run the auto-encoder reconstruction
            image_batch_1, image_batch_2, _ = next(loader)

            auto_encoder_optimizer.zero_grad()

            X_1.copy_(image_batch_1)
            X_2.copy_(image_batch_2)

            style_mu_1, style_logvar_1, class_latent_space_1 = encoder(Variable(X_1))
            style_latent_space_1 = reparameterize(training=True, mu=style_mu_1, logvar=style_logvar_1)

            kl_divergence_loss_1 = FLAGS.kl_divergence_coef * (
                - 0.5 * torch.sum(1 + style_logvar_1 - style_mu_1.pow(2) - style_logvar_1.exp())
            )
            kl_divergence_loss_1 /= (FLAGS.batch_size * FLAGS.num_channels * FLAGS.image_size * FLAGS.image_size)
            kl_divergence_loss_1.backward(retain_graph=True)

            style_mu_2, style_logvar_2, class_latent_space_2 = encoder(Variable(X_2))
            style_latent_space_2 = reparameterize(training=True, mu=style_mu_2, logvar=style_logvar_2)

            kl_divergence_loss_2 = FLAGS.kl_divergence_coef * (
                - 0.5 * torch.sum(1 + style_logvar_2 - style_mu_2.pow(2) - style_logvar_2.exp())
            )
            kl_divergence_loss_2 /= (FLAGS.batch_size * FLAGS.num_channels * FLAGS.image_size * FLAGS.image_size)
            kl_divergence_loss_2.backward(retain_graph=True)

            reconstructed_X_1 = decoder(style_latent_space_1, class_latent_space_2)
            reconstructed_X_2 = decoder(style_latent_space_2, class_latent_space_1)

            reconstruction_error_1 = FLAGS.reconstruction_coef * mse_loss(reconstructed_X_1, Variable(X_1))
            reconstruction_error_1.backward(retain_graph=True)

            reconstruction_error_2 = FLAGS.reconstruction_coef * mse_loss(reconstructed_X_2, Variable(X_2))
            reconstruction_error_2.backward()

            reconstruction_error = (reconstruction_error_1 + reconstruction_error_2) / FLAGS.reconstruction_coef
            kl_divergence_error = (kl_divergence_loss_1 + kl_divergence_loss_2) / FLAGS.kl_divergence_coef

            auto_encoder_optimizer.step()

            # B. reverse cycle
            image_batch_1, _, __ = next(loader)
            image_batch_2, _, __ = next(loader)

            reverse_cycle_optimizer.zero_grad()

            X_1.copy_(image_batch_1)
            X_2.copy_(image_batch_2)

            style_latent_space.normal_(0., 1.)

            _, __, class_latent_space_1 = encoder(Variable(X_1))
            _, __, class_latent_space_2 = encoder(Variable(X_2))

            reconstructed_X_1 = decoder(Variable(style_latent_space), class_latent_space_1.detach())
            reconstructed_X_2 = decoder(Variable(style_latent_space), class_latent_space_2.detach())

            style_mu_1, style_logvar_1, _ = encoder(reconstructed_X_1)
            style_latent_space_1 = reparameterize(training=False, mu=style_mu_1, logvar=style_logvar_1)

            style_mu_2, style_logvar_2, _ = encoder(reconstructed_X_2)
            style_latent_space_2 = reparameterize(training=False, mu=style_mu_2, logvar=style_logvar_2)

            reverse_cycle_loss = FLAGS.reverse_cycle_coef * l1_loss(style_latent_space_1, style_latent_space_2)
            reverse_cycle_loss.backward()
            reverse_cycle_loss /= FLAGS.reverse_cycle_coef

            reverse_cycle_optimizer.step()

            if (iteration + 1) % 10 == 0:
                print('')
                print('Epoch #' + str(epoch))
                print('Iteration #' + str(iteration))

                print('')
                print('Reconstruction loss: ' + str(reconstruction_error.data.storage().tolist()[0]))
                print('KL-Divergence loss: ' + str(kl_divergence_error.data.storage().tolist()[0]))
                print('Reverse cycle loss: ' + str(reverse_cycle_loss.data.storage().tolist()[0]))

            # write to log
            with open(FLAGS.log_file, 'a') as log:
                log.write('{0}\t{1}\t{2}\t{3}\t{4}\n'.format(
                    epoch,
                    iteration,
                    reconstruction_error.data.storage().tolist()[0],
                    kl_divergence_error.data.storage().tolist()[0],
                    reverse_cycle_loss.data.storage().tolist()[0]
                ))

            # write to tensorboard
            writer.add_scalar('Reconstruction loss', reconstruction_error.data.storage().tolist()[0],
                              epoch * (int(len(dataset) / FLAGS.batch_size) + 1) + iteration)
            writer.add_scalar('KL-Divergence loss', kl_divergence_error.data.storage().tolist()[0],
                              epoch * (int(len(dataset) / FLAGS.batch_size) + 1) + iteration)
            writer.add_scalar('Reverse cycle loss', reverse_cycle_loss.data.storage().tolist()[0],
                              epoch * (int(len(dataset) / FLAGS.batch_size) + 1) + iteration)

        # save model after every 5 epochs
        if (epoch + 1) % 5 == 0 or (epoch + 1) == FLAGS.end_epoch:
            torch.save(encoder.state_dict(), os.path.join('checkpoints', FLAGS.encoder_save))
            torch.save(decoder.state_dict(), os.path.join('checkpoints', FLAGS.decoder_save))

            
            

def plot_style_grid_matrix(FLAGS):


  encoder = Encoder(style_dim=FLAGS.style_dim, class_dim=FLAGS.class_dim)
   
  decoder = Decoder(style_dim=FLAGS.style_dim, class_dim=FLAGS.class_dim)
  """
  save reconstructed images and style swapped image generations to check progress
            """
  """
    variable definition
    """

  X_1 = torch.FloatTensor(FLAGS.batch_size, FLAGS.num_channels, FLAGS.image_size, FLAGS.image_size)
  X_2 = torch.FloatTensor(FLAGS.batch_size, FLAGS.num_channels, FLAGS.image_size, FLAGS.image_size)
 
  # load saved models if load_saved flag is true
  if FLAGS.load_saved:
      encoder.load_state_dict(torch.load(os.path.join('checkpoints', FLAGS.encoder_save)))
      decoder.load_state_dict(torch.load(os.path.join('checkpoints', FLAGS.decoder_save)))
  


  if FLAGS.cuda:
        encoder.cuda()
        decoder.cuda()

        

        X_1 = X_1.cuda()
        X_2 = X_2.cuda()
        
  
 
  image_batch_1, _, _ = next(test_loader)
  image_batch_2, _, _ = next(test_loader)
  
  
  X_1.copy_(image_batch_1)
  X_2.copy_(image_batch_2)
  

  style_mu_1, style_logvar_1, _ = encoder(Variable(X_1))
  _, __, class_latent_space_2 = encoder(Variable(X_2))
  

  style_latent_space_1 = reparameterize(training=False, mu=style_mu_1, logvar=style_logvar_1)
 
  # print(np.array(style_latent_space_1.detach().cpu()).shape, np.array(class_latent_space_2.detach().cpu()).shape)
  style_latent_nparray = np.array(style_latent_space_1.detach().cpu())
  # print(arr[0] == arr[1], arr[0] == arr[2])

  # reconstructed_X_1_2 = decoder(style_latent_space_1, class_latent_space_2)
  # reconstructed_X_3_2 = decoder(style_latent_space_3, class_latent_space_2)
  k = style_latent_nparray.shape[0]
  reconstructed_images = np.zeros((k, 60, 60, 3))
  for i in range(0, k):
    style_latent_temp = np.zeros((k, FLAGS.style_dim),dtype = 'float32')
    for j in range(0, k):
      style_latent_temp[j] = style_latent_nparray[i]

    
    if FLAGS.cuda:
      style_latent_temp = torch.from_numpy(style_latent_temp)
      style_latent_temp = style_latent_temp.cuda()
    reconstructed_x = decoder(style_latent_temp, class_latent_space_2)
    reconstructed_xx = np.transpose(reconstructed_x.cpu().data.numpy(), (0, 3, 2, 1))
    if i == 0:
      reconstructed_images = np.copy(reconstructed_xx)
    else:
      reconstructed_images = np.concatenate([reconstructed_images, reconstructed_xx], 0)
  
  image_batch_2 = np.array(image_batch_2)
  print(type(image_batch_2))
  image_batch_2 = np.transpose(image_batch_2, (0, 3, 2, 1))
  reconstructed_images = np.concatenate([image_batch_2, reconstructed_images], 0)
  print(reconstructed_images.shape)
  image_batch_1 = np.transpose(image_batch_1.numpy(), (0, 3, 2, 1))
  blank_img = np.zeros((60, 60, 3))
  image_list = []
  image_list.append(blank_img)
  j = 0
  for i in range(reconstructed_images.shape[0]):
    if i % 8 == 0 and j != 8 and i != 0:
      image_list.append(image_batch_1[j])
      
      j += 1
    image_list.append(reconstructed_images[i])
  print(len(image_list))
  fig = plt.figure(figsize=(12., 12.))
  grid = ImageGrid(fig, 111,  # similar to subplot(111)
                  nrows_ncols=(9, 9),  # creates 2x2 grid of axes
                  axes_pad=0.01,  # pad between axes in inch.
                  )
  # print(label1)
  for ax, im in zip(grid, image_list):
      # Iterating over the grid returns the Axes.
      ax.imshow(im)

  plt.show()

  # print(reconstructed_images[0] == reconstructed_images[255], reconstructed_images[0] == reconstructed_images[128])


      
def linear_interpolation_line(FLAGS):
  encoder = Encoder(style_dim=FLAGS.style_dim, class_dim=FLAGS.class_dim)   
  decoder = Decoder(style_dim=FLAGS.style_dim, class_dim=FLAGS.class_dim)
  
  X_1 = torch.FloatTensor(FLAGS.batch_size, FLAGS.num_channels, FLAGS.image_size, FLAGS.image_size)
  X_2 = torch.FloatTensor(FLAGS.batch_size, FLAGS.num_channels, FLAGS.image_size, FLAGS.image_size)
 
  # load saved models if load_saved flag is true
  if FLAGS.load_saved:
      encoder.load_state_dict(torch.load(os.path.join('checkpoints', FLAGS.encoder_save)))
      decoder.load_state_dict(torch.load(os.path.join('checkpoints', FLAGS.decoder_save)))
 
  if FLAGS.cuda:
        encoder.cuda()
        decoder.cuda()
        X_1 = X_1.cuda()
        X_2 = X_2.cuda() 
  image_batch_1, _ ,_  = next(test_loader)
  image_batch_2, _, _ = next(test_loader)
  
  X_1.copy_(image_batch_1)
  X_2.copy_(image_batch_2)

  style_mu_1, style_logvar_1, class_latent_space_1 = encoder(Variable(X_1))
  style_mu_2, style_logvar_2, class_latent_space_2 = encoder(Variable(X_2))
  
  style_latent_space_1 = reparameterize(training= False, mu = style_mu_1, logvar = style_logvar_1)
  style_latent_space_2 = reparameterize(training = False, mu = style_mu_2, logvar = style_logvar_2)
  image_batch_1 = np.transpose(image_batch_1.numpy(), (0, 3, 2, 1))
  image_batch_2 = np.transpose(image_batch_2.numpy(), (0, 3, 2, 1))
  style_latent_nparray1 = np.array(style_latent_space_1.detach().cpu())
  style_latent_nparray2 = np.array(style_latent_space_2.detach().cpu())
  class_latent_nparray2 = np.array(class_latent_space_2.detach().cpu())
  class_latent_nparray1 = np.array(class_latent_space_1.detach().cpu())
  reconstructed_images = np.zeros((8, 8, 60, 60, 3), dtype = 'float32')
  class_latent_spaces = np.zeros((8, 8, 1, 64), dtype = 'float32')
  style_latent_spaces = np.zeros((8, 8, 1, 64), dtype = 'float32')
  steps = 8
  for k in range(8):
    new_class_latent_space = (class_latent_nparray2 - class_latent_nparray1) *  k / steps + class_latent_nparray1
    style_latent_spaces[0][k] = style_latent_nparray1
    class_latent_spaces[0][k] = new_class_latent_space
    # x = torch.from_numpy(new_class_latent_space)
    # class_latent_space = x.cuda()

    
    # recons = decoder(style_latent_space_1, class_latent_space)
    # print(recons)
    # reconstructed_images[k] = np.transpose(recons.detach().cpu().data.numpy(), (0, 3, 2, 1))[0]
  for k in range(7, -1, -1):
    new_class_spaces = (class_latent_nparray1 - class_latent_nparray2) * (7 - k)/steps + class_latent_nparray2
    style_latent_spaces[7][k] = style_latent_nparray2
    class_latent_spaces[7][k] = new_class_spaces
  steps = 6
  for k in range(0, 8):
    for j in range(1, 7):
      new_style_spaces = (style_latent_spaces[7][k] - style_latent_spaces[0][k])  * j / steps + style_latent_spaces[0][k]
      style_latent_spaces[j][k] = new_style_spaces
      class_latent_spaces[j][k] = class_latent_spaces[j][7]
  for j in range(0, 8):
    for k in range(0, 8):
      # print(np.shape(style_latent_spaces[j][k]))
      stl = torch.from_numpy(style_latent_spaces[j][k])
      cls = torch.from_numpy(class_latent_spaces[j][k])
      stl = stl.cuda()
      cls = cls.cuda()
      img = decoder(stl, cls)
      reconstructed_images[j][k] = np.transpose(img.detach().cpu().data.numpy(), (0, 3, 2, 1))[0]
      # print(np.shape(reconstructed_images[j][k]))
  # print(np.shape(image_batch_1[0]))
  reconstructed_images[0][0] = image_batch_1[0]
  reconstructed_images[7][7] = image_batch_2[0]


  image_list = []
  
  for i in range(reconstructed_images.shape[0]):
    for j in range(reconstructed_images.shape[1]):
      image_list.append(reconstructed_images[i][j])
    
  # print(len(image_list))
  fig = plt.figure(figsize=(10., 10.))
  grid = ImageGrid(fig, 111,  # similar to subplot(111)
                  nrows_ncols=(8, 8),  # creates 2x2 grid of axes
                  axes_pad=0.01,  # pad between axes in inch.
                  )
  # print(label1)
  for ax, im in zip(grid, image_list):
      # Iterating over the grid returns the Axes.
      ax.imshow(im)

  plt.show()

In [0]:

F1 = ags()
F1.load_saved = True
F1.batch_size = 1
linear_interpolation_line(F1)

In [0]:
F2 = ags()
F2.load_saved = True
F2.batch_size = 8
plot_style_grid_matrix(F2)

In [0]:
encoder = Encoder(64, 64)
encoder.parameters()

In [0]:
import numpy as np
np.arange(1, 8)